In [ ]:
import os
import pandas as pd
import argparse
import re
import lxml.etree as ET
from xml_ns import ns
from vocab_map_file import oid_map
import vocab_maps
import create_map_to_standard

In [ ]:
#Load Foundry Datasets
from foundry.transforms import Dataset
omop_concept = Dataset.get("concept").read_table(format="pandas")
omop_concept_relationship = Dataset.get("concept_relationship").read_table(format="pandas")


In [ ]:
# Load and parse the XML file
with open('./resources/Patient-502.xml', 'rb') as file:
    tree = etree.parse(file)

# Define namespaces to use when searching
ns = {'hl7': 'urn:hl7-org:v3'}

# Directory containing the XML files
directory_path = './resources/'

# Define namespaces to use when searching
ns = {'hl7': 'urn:hl7-org:v3'}

# Initialize a list to store records for DataFrame
data_records = []

In [ ]:
    print("Reading Vocab, takes a minute")
    concept_df = vocab_maps.read_concept()

In [ ]:
 # get vocabularies
(oid_map_df, concept_df, concept_relationship_df,source_to_concept_df) = vocab_maps.read_vocabulary_tables()

In [ ]:


# Iterate over all XML files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xml'):
        file_path = os.path.join(directory_path, filename)

        # Load and parse the XML file
        try:
            with open(file_path, 'rb') as file:
                tree = ET.parse(file)

            # Define elements to extract
            elements = [
                ('administrativeGenderCode', 'Gender'),
                ('raceCode', 'Race'),
                ('ethnicGroupCode', 'Ethnicity'),
                ('birthTime', 'BirthTime')
            ]
            
            # Extract data for each element
            record = {'Filename': filename}
            for tag, label in elements:
                element = tree.find(f'.//hl7:{tag}', namespaces=ns)
                value = element.get('displayName') if tag != 'birthTime' and element is not None else (
                    element.get('value') if element is not None else 'Unknown'
                )
                record[label] = value

            # Append data to the records list
            data_records.append(record)

        except Exception as e:
            print(f"Failed to parse {filename}: {e}")

# Create DataFrame from the records
ccda_snooper_people = pd.DataFrame(data_records)

# Print the DataFrame
print(ccda_snooper_people)

# Optionally, save the DataFrame to a CSV file
#df.to_csv('output_summary_with_loop.csv', index=False)


In [ ]:
from foundry.transforms import Dataset

dq_ccda_snooper_people = Dataset.get("dq_ccda_snooper_people")
dq_ccda_snooper_people.write_table(ccda_snooper_people)